# Diffie-Hellman-Merkel Key Exchange Demo
## Introduction
Most end-to-end encryption/secure transfer protocols which you have been introduced to uptil now work on the assumption that there is a way to securely obtain a pair of symmetric keys, one each side. Diffie-Hellman-Merkel Key Exchange is one of the earliest ways of achieving this functionality without really ever transporting the actual keys between the devices.

[This figure](https://en.wikipedia.org/wiki/File:Diffie-Hellman_Key_Exchange.svg) from [Wikipedia](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange) does a good job of explaining the simplest implementation of DHKE. The 'common paint' stands for the large prime number *n* which is assumed to be in public domain. Secret colors indicate the private keys of the individual machines (these never leave the local storage). The first product, represented by the pale orange and pale blue solutions respectively is the public generator *g* raised to the power of the private keys of the individual machines. While the second product, the DHKey is the first product of the second machine raised to the private key of the first and vice-versa respectively essentially creating identical keys on both devices without exposing either devices' private key over the network.

## Approach
Instead of creating an API and running two parallel servers to simulate the communication between two machines, we will instead programmatically transfer data between two instances of the Machine class.

In [ ]:
!pip install pycrypto

In [80]:
from Crypto.Util.number import getPrime, getRandomNBitInteger
from time import time

In [81]:
# Lengths in bits
LENGTH_MACHINE_PRIVATE_KEY = 8
LENGTH_PUBLIC_GENERATOR = 4
LENGTH_PUBLIC_PRIME = 1024

In [82]:
#Available in public domain - maybe stashed or generated in realtime
PUBLIC_GENERATOR = getRandomNBitInteger(LENGTH_PUBLIC_GENERATOR) #g
PUBLIC_PRIME = getPrime(LENGTH_PUBLIC_PRIME) #n
print("Public Generator: {g} \nPublic Prime: {n}".format(g=PUBLIC_GENERATOR, n=PUBLIC_PRIME))

Public Generator: 10 
Public Prime: 177268008635034070066391028824253078138170101521172570890908216933331993459250667087781212722911604955978135210005319872891292379839599815763745847440415020631800267824640454214901912468701140019317694027739638789051735271026970112679327933729727194306967241142181255278035238702583911799599977500505255210383


In [83]:
def main():
  
  # to time the functions
  # merely to show difference in processing power required at different lengths of constants
  # no real life significance
  time_m1, time_m2 = 0, 0

  start_m1 = time()
  m1 = Machine() # Intializing simulation of first machine
  k1 = m1.getPublicKey(PUBLIC_GENERATOR, PUBLIC_PRIME) # obtaining public key generated by the first machine
  end_m1 = time()
  time_m1 = end_m1 - start_m1

  priv_k1 = m1.getPrivateKey() # obtaining private key of the first machine (only for demo)

  start_m2 = time()
  m2 = Machine() # Intializing simulation of second machine
  k2 = m2.getPublicKey(PUBLIC_GENERATOR, PUBLIC_PRIME)  # obtaining public key generated by the second machine
  end_m2 = time()
  time_m2 = end_m2 - start_m2

  priv_k2 = m2.getPrivateKey() # obtaining private key of the second machine (only for demo)

  start_m1 = time()
  dhk1 = m1.setDHKey(k2) # obtaining DHKey for the first machine. In real life this would just be stored on that machine locally
  end_m1 = time()
  time_m1 += end_m1 - start_m1

  start_m2 = time()
  dhk2 = m2.setDHKey(k1) # obtaining DHKey for the second machine. In real life this would just be stored on that machine locally
  end_m2 = time()
  time_m2 += end_m2 - start_m2

  print(f"Machine M1:\nPrivate Key (priv_k1): {priv_k1}\nPublic Key (k1): {k1}\nDH Key (dhk1): {dhk1}\nTime taken: {time_m1}\n\n")
  print(f"Machine M2:\nPrivate Key (priv_k2): {priv_k2}\nPublic Key (k2): {k2}\nDH Key (dhk2): {dhk2}\nTime taken: {time_m2}\n\n")

In [84]:
class Machine:
  def __init__(self):
    self.__private_key = getRandomNBitInteger(LENGTH_MACHINE_PRIVATE_KEY)
    self.__dh_key = 0
    self.__public_prime = None
  
  def getPublicKey(self, public_generator: int, public_prime: int) -> int:
    '''
    Returns public key after calculation of g^(a)mod(n) where 'a' is the private key of the machine

    :param int public_generator: generator 'g' from the public domain
    :param int public_prime: large prime number 'n' used as modulo
    :return: int public key
    '''
    self.__public_prime = public_prime
    return (pow(public_generator, self.__private_key)%self.__public_prime)

  def setDHKey(self, other_public_key: int) -> int:
    '''
    Sets the final DHKey after calculation of q^(a)mod(n) where 'q' is the public key of the other machine and 'a' is the private key of this machine
    Returns (only for demo) the final DHKey (this would just be stored in the local storage in real life)

    :param int other_public_key: public key generated by other machine (public key is the return value of Machine.getPublicKey()
    :return: int DHKey
    '''
    self.__dh_key = (pow(other_public_key, self.__private_key)%self.__public_prime)
    return self.__dh_key # do not transmit the dhkey in real life
  
  def getPrivateKey(self) -> int:
    '''
    Returns (only for demo) the private key of this machine. (This would never leave the machine in real life)
    '''
    return self.__private_key # do not transmit the private key in real life

In [85]:
main()

Machine M1:
Private Key (priv_k1): 231
Public Key (k1): 1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
DH Key (dhk1): 160300236002135923267565408864983074648681569761720782623552342338054444972700206415091398252885243324041721598293678380398079995607874262029957626174669772210302545293729089279747283698728348893026649523591379339640644834822222340483551888073477127062781621046562773364094657422408625326045722434468016840924
Time taken: 0.003507852554321289


Machine M2:
Private Key (priv_k2): 181
Public Key (k2): 10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
DH Key (dhk2): 1603002360021359232675654088649830746486815697617207826235523423380544449727002064150913982528852